id:    Unique id of news article,
title: The title of news article, 
author:Author of news article,
text:  Text of news article, it could be incomplete,
label: Represent the news article is real or fake ,

1: Fake news,
0: Real news.

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics import accuracy_score

# stopword are those words that does not add much values in paragraph(datasets) like "a, an, the, this, those, that what, where, in, at and many more"

# nltk stand for natural language toolkit 
#basically stemming takes aword and remove prefix and postfix from it and return root word
#TfidfVectorizer convert the text to feature vactors(numbers)

In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
np.array(stopwords.words("english"))

array(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
       "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
       'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
       'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
       'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
       'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
       'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
       'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
       'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
       'by', 'for', 'with', 'about', 'against', 'between', 'into',
       'through', 'during', 'before', 'after', 'above', 'below', 'to',
       'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
       'again', 'further', 'then', 'once', 'here', 'there', 'when',
       'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'm

In [4]:
df= pd.read_csv("news.csv")

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
print(df.label.unique())
print(df.label.nunique())
print(df.label.count())
print(df.label.values)

[1 0]
2
20800
[1 0 1 ... 0 1 1]


In [23]:
# df.dropna(inplace= True)
df.drop('id', axis=1, inplace=True)
# df.drop(columns='id', inplace= True)


In [11]:
# Replacing the null value with empty string

df= df.fillna("")

In [12]:
# Merging the Author name and new title in new column "content"

df["content"] =df["author"]+ " " + df["title"]

In [13]:
# separating Features and prediction

x =df.drop(columns='label')
y=df.label

In [14]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [15]:
# Stemming : stemming is the process of reducing a word to its Root Word
# Example : actor, actress, acting --> Act is root word

port_stem =PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content =re.sub('[^a-zA-Z]'," ", content)
    stemmed_content =stemmed_content.lower()
    stemmed_content =stemmed_content.split()
    stemmed_content =[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content =" ".join(stemmed_content)
    return stemmed_content 


# 1st step: Replaced all the number or sign to a space 
# 2nd step: Change all characher in lower case
# 3rd step: All character in 1st step has space , To remove than spaces and converted it to List
# 4th step: 1st-- Take that word and stemmed them and convert into root word
# 4th step: 2nd-- And removing all the stopwords from the list(this step run first)
# 5th step: Join all the words.

In [17]:
stemming("I love this programm may i buy 277 tickets to your show in@@ you are great actor and actress. oing great acting no one can do that type of performance")

'love programm may buy ticket show great actor actress o great act one type perform'

In [18]:
df['content'] = df['content'].apply(stemming)

In [19]:
df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exercis ...


In [29]:
# Again separate the data and label (The first one X is very time consuming because of large data)

x= df.content.values
y=df.label.values

In [30]:
# The content has textual data , computer does not understand textual data so
# Converting the Textual data to Numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(x)

# Tf stands for Term Frequency and idf transfer Inverse Document frequence
# tf: That count the number of a word repeatation
# idf: Sometimes repeated words does not gave any meaning in it

x = vectorizer.transform(x)
# Transform it so that our model will understand it.

In [31]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [32]:
# Splitting the dataset to training and testing data

x_train, x_test, y_train, y_test = train_test_split(x,y , test_size=0.2, stratify=y, random_state=2)

# stratify represent y, It segregated in a equal propostion 

In [34]:
# model Selection 
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [35]:
y_pred = model.predict(x_test)

In [36]:
# Find Accuracy of our test data 
accuracy_score(y_test, y_pred)*100

97.90865384615385

In [40]:
# Making prediction system

x_new = x_test[0]
prediction = model.predict(x_new)

if prediction[0]==0:
    print("The news is Real")
else:
    print("The news is Fake")
print(prediction[0])

The news is Fake
1


In [41]:
y_test[0]

1

In [45]:
prediction[0] == y_test[0]

True